In [1]:
import pandas as pd
from datetime import datetime, date

In [2]:
tabc_gr = pd.read_csv('/Users/michaelhinds/Downloads/Mixed_Beverage_Gross_Receipts.csv')

In [3]:
tabc_gr.dtypes

Taxpayer Number                 int64
Taxpayer Name                  object
Taxpayer Address               object
Taxpayer City                  object
Taxpayer State                 object
Taxpayer Zip                  float64
Taxpayer County                 int64
Location Number                 int64
Location Name                  object
Location Address               object
Location City                  object
Location State                 object
Location Zip                    int64
Location County                 int64
Inside/Outside City Limits     object
TABC Permit Number             object
Responsibility Begin Date      object
Responsibility End Date        object
Obligation End Date            object
Liquor Receipts                 int64
Wine Receipts                   int64
Beer Receipts                   int64
Cover Charge Receipts           int64
Total Receipts                  int64
dtype: object

In [5]:
tabc_gr.head()

,Taxpayer Number,Taxpayer Name,Taxpayer Address,Taxpayer City,Taxpayer State,Taxpayer Zip,Taxpayer County,Location Number,Location Name,Location Address,...,Inside/Outside City Limits,TABC Permit Number,Responsibility Begin Date,Responsibility End Date,Obligation End Date,Liquor Receipts,Wine Receipts,Beer Receipts,Cover Charge Receipts,Total Receipts
0,32047970895,HONDURAS MAYA CAFE & BAR LLC,8011 HAZEN ST,HOUSTON,TX,77036.0,101,1,HONDURAS MAYA CAFE & BAR LLC,5945 BELLAIRE BLVD STE B,...,Y,MB817033,08/16/2012,09/12/2019,07/31/2019,0,0,0,0,0
1,32049923835,"MERMAID KARAOKE PRIVATE CLUB, INC.",2639 WALNUT HILL LN STE 225,DALLAS,TX,75229.0,57,1,MERMAID KARAOKE PRIVATE CLUB,1310 W CAMPBELL RD STE 103,...,Y,N 837378,04/12/2013,07/01/2015,08/31/2014,480,185,1374,0,2039
2,32034036304,FENG KAI CORPORATION,8427 BOULEVARD 26,N RICHLND HLS,TX,76180.0,220,1,JAPANESE GRILL,8427 BOULEVARD 26,...,Y,MB576670,05/01/2008,03/17/2018,06/30/2016,1143,167,669,0,1979
3,14537211071,"THE HUTTO SMITHS, LLC",429 LITTLE LAKE RD,HUTTO,TX,78634.0,246,1,THE DOWNTOWN HALL OF FAME,205 EAST ST,...,Y,MB791778,12/06/2011,NaN,03/31/2018,12881,357,10447,0,23685
4,32019999229,"THE CROSSING AT FIDDLE CREEK, INC.",1620 W CEDAR ST,STEPHENVILLE,TX,76401.0,72,1,THE CROSSING AT FIDDLE CREEK INC,2004 W SWAN ST,...,Y,N 643163,10/26/2006,11/30/2013,08/31/2008,4841,2413,4620,0,11874


In [6]:
tabc_gr2 = tabc_gr.drop_duplicates()

In [11]:
len(tabc_gr) == len(tabc_gr2)

True

In [12]:
len(tabc_gr['TABC Permit Number'].unique())

42174

In [13]:
tabc_gr[tabc_gr['TABC Permit Number'] == 'MB817033'].iloc[0]

Taxpayer Number                                32047970895
Taxpayer Name                 HONDURAS MAYA CAFE & BAR LLC
Taxpayer Address                             8011 HAZEN ST
Taxpayer City                                      HOUSTON
Taxpayer State                                          TX
Taxpayer Zip                                       77036.0
Taxpayer County                                        101
Location Number                                          1
Location Name                 HONDURAS MAYA CAFE & BAR LLC
Location Address                  5945 BELLAIRE BLVD STE B
Location City                                      HOUSTON
Location State                                          TX
Location Zip                                         77081
Location County                                        101
Inside/Outside City Limits                               Y
TABC Permit Number                                MB817033
Responsibility Begin Date                       08/16/20

In [16]:
def remove_dup_tabc_permit_numbers(df):
    df['date'] = pd.to_datetime(df['Obligation End Date'], format='%m/%d/%Y')
    df_obs = df.groupby(by=['TABC Permit Number', 'date']).size()
    yeet_list = [x[0] for x in df_obs[df_obs > 1].index]
    return yeet_list

In [19]:
tabc_gr['date'] = pd.to_datetime(tabc_gr['Obligation End Date'], format='%m/%d/%Y')

In [30]:
tabc_obs = tabc_gr.groupby(by=['TABC Permit Number', 'date']).size()

In [31]:
tabc_taxpayer_obs = tabc_gr.groupby(by=['TABC Permit Number', 'Taxpayer Number',  'date']).size()

In [43]:
tabc_tax_name_obs = tabc_gr.groupby(by=['TABC Permit Number', 'Taxpayer Number', 'Location Name', 'date']).size()

In [68]:
tabc_tax_name_add_obs = tabc_gr.groupby(by=['TABC Permit Number',
                                            'Taxpayer Number',
                                            'Location Name',
                                            'Location Address',
                                            'date']).size()

In [86]:
new_index_df = tabc_tax_name_add_obs.reset_index(drop=False)

In [87]:
new_index_df = new_index_df.drop(columns=new_index_df.columns[-2:])

In [91]:
new_index_df = new_index_df.drop_duplicates().reset_index(drop=True)

In [97]:
new_index_df['TABC Permit Number'].nunique()

42174

In [98]:
new_index_df['Taxpayer Number'].nunique()

34824

In [99]:
new_index_df['Location Name'].nunique()

35464

In [100]:
new_index_df['Location Address'].nunique()

31843

In [101]:
new_index_df.shape

(46280, 4)

In [105]:
from hashlib import sha1

In [107]:
new_index_df['id'] = new_index_df\
                        .apply(lambda x: ''.join(x.astype(str)),axis=1)\
                        .apply(lambda value: sha1(str(value).encode('utf-8')).hexdigest())

In [111]:
new_index_df.to_csv('bar_id.csv')

In [60]:
tabc_tax_name_add_obs.describe()

count    2.788483e+06
mean     1.000006e+00
std      2.540686e-03
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.000000e+00
dtype: float64

In [61]:
tabc_tax_name_add_obs[tabc_tax_name_add_obs > 1].shape

(18,)

In [46]:
tabc_tax_name_obs[tabc_tax_name_obs == 2]

TABC Permit Number  Taxpayer Number  Location Name                     date      
D 805363            32045781765      GENIUS LIQUIDS LLC                2020-03-31    2
D 891663            32049107785      KOOPER FAMILY WHISKEY COMPANY     2018-05-31    2
D 927360            32057617097      STINSON DISTILLING                2021-01-31    2
D 937876            32058443857      AUSTONIAN SPIRITS, INC.           2018-08-31    2
D 942488            32010724683      SANGIOVANNI DISTILLERY            2019-10-31    2
                                                                                    ..
RM985290            32062989192      DRAKES YOKE WOOD & WINE           2017-09-30    2
SL782206            32053652486      POKES                             2013-08-31    2
TB000001            17602973145      CHRISTMAS PARTY                   2007-12-31    2
TB012345            17501130672      KHIVA MULE SKINNERS - FUNDRAISER  2010-12-31    2
TB017844            30116323541      BRAZOS VALL

In [45]:
tabc_tax_name_obs.describe()

count    2.787982e+06
mean     1.000186e+00
std      1.364264e-02
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.000000e+00
dtype: float64

In [ ]:
D 805363

In [34]:
tabc_taxpayer_obs[tabc_taxpayer_obs == 3]

TABC Permit Number  Taxpayer Number  date      
MB686563            12615655813      2011-02-28    3
TB000001            17602973145      2007-12-31    3
dtype: int64

In [33]:
tabc_taxpayer_obs.describe()

count    2.786712e+06
mean     1.000642e+00
std      2.535742e-02
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      3.000000e+00
dtype: float64

In [28]:
tabc_obs[tabc_obs == 5]

TABC Permit Number  date      
TB000001            2007-12-31    5
dtype: int64

In [23]:
tabc_obs.describe()

count    2.785339e+06
mean     1.001135e+00
std      3.381237e-02
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      5.000000e+00
dtype: float64

In [49]:
!pwd

/Users/michaelhinds/Documents/APDS/session_18Aug2022/texas_beer


In [54]:
tabc_gr[(tabc_gr['TABC Permit Number'] == 'MB192118')].sort_values(by='date').to_csv('dups2.csv')

In [29]:
tabc_gr[(tabc_gr['TABC Permit Number'] == 'TB000001')]

,Taxpayer Number,Taxpayer Name,Taxpayer Address,Taxpayer City,Taxpayer State,Taxpayer Zip,Taxpayer County,Location Number,Location Name,Location Address,...,TABC Permit Number,Responsibility Begin Date,Responsibility End Date,Obligation End Date,Liquor Receipts,Wine Receipts,Beer Receipts,Cover Charge Receipts,Total Receipts,date
1051143,32015503777,TOMBALL CENTENNIAL COMMITEE,PO BOX 2007,TOMBALL,TX,77377.0,101,2,FUNDRAISER,824 ULRICH RD,...,TB000001,12/01/2007,12/01/2007,06/30/2007,2555,0,33912,0,36467,2007-06-30
1190500,17602973145,"SUNSET G VENTURES, INC.",1702 WILSON RD,HUMBLE,TX,77396.0,101,7,FESTIVAL OF GIVING,1201 LAKE WOODLANDS DR,...,TB000001,12/01/2007,12/31/2007,12/31/2007,0,528,15,0,543,2007-12-31
1199962,17603353867,"KREWE OF OLYMPUS - TEXAS, INC.",PO BOX 920794,HOUSTON,TX,77292.0,101,15,FUNDRAISER,412 LOVETT BLVD,...,TB000001,10/01/2007,10/01/2007,10/31/2007,1044,114,417,0,1575,2007-10-31
1259382,17602973145,"SUNSET G VENTURES, INC.",1702 WILSON RD,HUMBLE,TX,77396.0,101,8,CHRISTMAS PARTY,3100 WESLAYAN ST STE 400,...,TB000001,12/01/2007,12/01/2007,12/31/2007,353,15,47,0,415,2007-12-31
1380111,17602973145,"SUNSET G VENTURES, INC.",1702 WILSON RD,HUMBLE,TX,77396.0,101,10,CHAMBER OF COMMERCE,21575 HIGHWAY 59 N,...,TB000001,01/01/2008,01/01/2008,01/31/2008,460,48,36,0,544,2008-01-31
1428014,32015503777,TOMBALL CENTENNIAL COMMITEE,PO BOX 2007,TOMBALL,TX,77377.0,101,2,FUNDRAISER,824 ULRICH RD,...,TB000001,12/01/2007,12/01/2007,12/31/2007,0,0,3204,0,3204,2007-12-31
1451535,17602973145,"SUNSET G VENTURES, INC.",1702 WILSON RD,HUMBLE,TX,77396.0,101,6,CHRISTMAS PARTY,8233 WILL CLAYTON PKWY,...,TB000001,12/01/2007,12/01/2007,12/31/2007,356,84,274,0,714,2007-12-31
1746359,17600712982,BLUEBONNET SOCIETY OF BELLVILLE,PO BOX 92,BELLVILLE,TX,77418.0,8,1,ANNUAL FUNDRAISER DINNER AND DANCE,FAIRGROUNDS AUSTIN CO EXHIBIT BLDG,...,TB000001,04/01/2007,04/01/2007,04/30/2007,2875,0,0,0,2875,2007-04-30
2660021,17501130672,KHIVA SHRINE TEMPLE,PO BOX 328,AMARILLO,TX,79105.0,188,16,MULE SKINNER'S FUNDRAISER,305 SE 5TH AVE,...,TB000001,12/01/2007,12/01/2007,09/30/2007,770,0,480,0,1250,2007-09-30
2663162,17501130672,KHIVA SHRINE TEMPLE,PO BOX 328,AMARILLO,TX,79105.0,188,16,MULE SKINNER'S FUNDRAISER,305 SE 5TH AVE,...,TB000001,12/01/2007,12/01/2007,12/31/2007,520,42,0,0,562,2007-12-31


In [16]:
tabc_lics = pd.read_csv('/Users/michaelhinds/Downloads/TABCLicenses.csv')

/var/folders/4c/2qnjjn4s5ll345918sr_tf8r0000gn/T/ipykernel_78543/2791111716.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  tabc_lics = pd.read_csv('/Users/michaelhinds/Downloads/TABCLicenses.csv')


In [17]:
tabc_lics.dtypes

AIMSOwnerName           object
AIMSTradeName           object
AIMSLicenseType         object
AIMSLicenseID            int64
LegacyLicenseClass      object
LegacyLicenseNumber    float64
LocationAddress         object
City                    object
State                   object
ZIP                     object
TXCounty                object
dtype: object

In [18]:
tabc_lics.head()

,AIMSOwnerName,AIMSTradeName,AIMSLicenseType,AIMSLicenseID,LegacyLicenseClass,LegacyLicenseNumber,LocationAddress,City,State,ZIP,TXCounty
0,"""BUBBA'S """"THE"""" BOILING SPOT INC.""",BUBBA'S,BG,102548510,BG,756557.0,1142 HIGHWAY 185 E,SEADRIFT,TX,77983,Calhoun
1,"""CIRCLE """"ON"""" INC""","""CIRCLE """"ON""""""",BQ,103435005,BQ,831979.0,207 S CLARK ROAD,CEDAR HILL,TX,751042773,Dallas
2,"""DAVID """"EL KID"""" MEAT MARKET INC.""",DAVID'S MEAT MARKET,BQ,101677104,BQ,688572.0,3310 E BARRERA ST,MERCEDES,TX,785705738,Hidalgo
3,"""DAVID """"EL KID"""" MEAT MARKET INC.""",EL KID'S DRIVE THRU & MEATS TO GO,BQ,103868418,BQ,867249.0,808 E MILE 10 N,WESLACO,TX,785992321,Hidalgo
4,"""JR. AND POLEY""""S SPOT LLC""",JR AND POLEY''S SPOT LLC,BG,106287318,BG,1062386.0,207 N OAK ST,PECOS,TX,797722336,Reeves


In [22]:
tabc_lics.LocationAddress

0                    1142 HIGHWAY 185 E  
1                      207 S CLARK ROAD  
2                     3310 E BARRERA ST  
3                       808 E MILE 10 N  
4                          207 N OAK ST  
                       ...               
66036    18900 N HEATHERWILDE BOULEVARD  
66037    2625 OLD DENTON ROAD SUITE 586  
66038                3026 FM 2094 STE A  
66039                   2628 N STORY RD  
66040              8305 ALMEDA GENOA RD  
Name: LocationAddress, Length: 66041, dtype: object

In [21]:
len('102548510')

9